# **Optimization**

In [30]:
import numpy as np
import cvxpy as cp
import cplex
from tqdm import tqdm

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [31]:
market_data = pd.read_csv('./new_market.csv')

In [32]:
stocks_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

stocks_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

stocks_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [33]:
def make_dict(dict):
    dic = {}
    
    for column in dict.keys():
        dic[column] = 1/len(dict.keys())

    return dic
    
gpt15_same = make_dict(stocks_15)
gpt30_same = make_dict(stocks_30)
gpt45_same = make_dict(stocks_45)

In [34]:
market_data['pricingDate'] = pd.to_datetime(market_data['pricingDate'])
market_data.set_index('pricingDate', inplace = True)

In [35]:
# yfinance에서 데이터 가져오는 것 대신 S&P market 데이터에서 데이터 가져울 수 있게 함수 만듬
# 원하는 티커(dict에서 가져와도됨), 날짜 넣으면 됨


def download(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = market_data.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

## **GMV**

In [36]:
# GMV 함수

def gmv(dict):

    data = download(dict, "2018-05-01", "2023-04-30")
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    cov_mat = np.cov(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Covariance Matrix를 만듬

    print(cov_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, cov_mat)) # 목적식 설정
    
    min_weights = 0.5 * 1 / num_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / num_assets
    
    # weights >= 0 
    const = [cp.sum(weights) == 1, 
             weights >= min_weights,
             weights <= max_weights]
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt

In [37]:
gmv_15 = gmv(stocks_15)
gmv_30 = gmv(stocks_30)
gmv_45 = gmv(stocks_45)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 284.32it/s]

[[4.41037202e-04 3.11416941e-04 4.56957152e-04 1.96735160e-04
  1.22198665e-04 1.18605250e-04 1.10754118e-04 1.18747228e-04
  1.52436256e-04 1.55920085e-04 2.92086610e-04 3.13770145e-04
  4.14083528e-04 2.37882047e-04 2.18709955e-04]
 [3.11416941e-04 5.18839538e-04 4.76619129e-04 1.44018837e-04
  8.38146215e-05 8.89556229e-05 7.64995103e-05 7.46461162e-05
  1.02478738e-04 1.21561958e-04 3.13022091e-04 3.18477206e-04
  4.06046654e-04 2.25399032e-04 1.70564136e-04]
 [4.56957152e-04 4.76619129e-04 1.08428481e-03 2.57219197e-04
  1.30863412e-04 1.32862108e-04 1.09130963e-04 1.20427898e-04
  1.96091129e-04 1.96161586e-04 4.27710755e-04 4.55466597e-04
  6.67841385e-04 3.54548567e-04 2.93065594e-04]
 [1.96735160e-04 1.44018837e-04 2.57219197e-04 4.12936364e-04
  1.07653580e-04 1.26055175e-04 1.08684208e-04 1.46753648e-04
  2.60851413e-04 1.31052286e-04 1.87814243e-04 1.85443252e-04
  2.31775817e-04 2.15066338e-04 4.19917148e-04]
 [1.22198665e-04 8.38146215e-05 1.30863412e-04 1.07653580e-04
  

Optimal


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 342.66it/s]

[[3.82966793e-04 3.18477206e-04 4.55466597e-04 3.13770145e-04
  3.11268538e-04 3.68585768e-04 1.10613902e-04 1.20695713e-04
  1.11200634e-04 2.01776048e-04 1.26016251e-04 1.18318831e-04
  1.85443252e-04 2.12512527e-04 1.67009341e-04 1.71387992e-04
  1.34610516e-04 2.59757553e-04 3.69139789e-04 1.59690246e-04
  2.29236607e-04 7.63056240e-05 3.41106747e-04 1.88669476e-04
  1.15848695e-04 3.10535632e-04 2.07342216e-04 2.42471212e-04
  1.44805305e-04 2.91109100e-04]
 [3.18477206e-04 5.18839538e-04 4.76619129e-04 3.11416941e-04
  3.13022091e-04 3.63007130e-04 7.64995103e-05 8.89556229e-05
  6.73221784e-05 1.68803584e-04 8.38146215e-05 7.46461162e-05
  1.44018837e-04 1.90112689e-04 1.46945638e-04 1.13621150e-04
  1.02478738e-04 2.26576181e-04 4.06046654e-04 1.21561958e-04
  2.25399032e-04 5.50775352e-05 3.52185410e-04 1.34911400e-04
  1.04121817e-04 2.98802474e-04 1.70564136e-04 2.06111212e-04
  9.78116463e-05 2.47418257e-04]
 [4.55466597e-04 4.76619129e-04 1.08428481e-03 4.56957152e-04
  4.

Optimal


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 346.61it/s]

[[4.41037202e-04 2.92086610e-04 3.13770145e-04 ... 2.20671774e-04
  7.67909847e-05 1.04148079e-04]
 [2.92086610e-04 4.01557685e-04 3.11268538e-04 ... 1.99290126e-04
  7.22217682e-05 9.08074477e-05]
 [3.13770145e-04 3.11268538e-04 3.82966793e-04 ... 2.20977700e-04
  7.63056240e-05 1.11200634e-04]
 ...
 [2.20671774e-04 1.99290126e-04 2.20977700e-04 ... 3.26766395e-04
  8.51788984e-05 9.67839738e-05]
 [7.67909847e-05 7.22217682e-05 7.63056240e-05 ... 8.51788984e-05
  1.54810218e-04 7.83539393e-05]
 [1.04148079e-04 9.08074477e-05 1.11200634e-04 ... 9.67839738e-05
  7.83539393e-05 2.12081322e-04]]


Optimal


In [38]:
# 누적 수익률 함수
# bool에 True를 넣으면 gpt gmv opt 데이터로 실행, False이면 GPT로 이미 가져와있는거 실행
def MDD(cumulative_returns):
    peak = cumulative_returns.iloc[0]
    drawdown = np.zeros(len(cumulative_returns))
    for i in range(1, len(cumulative_returns)):
        if cumulative_returns.iloc[i] > peak:
            peak = cumulative_returns.iloc[i]
        drawdown[i] = (peak - cumulative_returns.iloc[i]) / peak
    max_drawdown = np.max(drawdown)
    return max_drawdown

def STD(returns):
    std_dev = np.std(returns)
    return std_dev

def SHARPE(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    mean_excess_return = np.mean(excess_returns)
    std_dev = np.std(returns)
    sharpe_ratio = mean_excess_return / std_dev if std_dev != 0 else 0
    return sharpe_ratio

def cum_ret_gmv(dic, w_opt, start_date, end_date):  
    
    result = dict(zip(list(dic.keys()), w_opt))
    data = download(dic, start_date, end_date)
    ret = data.pct_change().dropna()
    pfo_ret = ret.dot(pd.Series(result))
    cum_ret = (1 + pfo_ret).cumprod()

    print(f'포트폴리오의 MDD는: {round(MDD(cum_ret),2)*100}%')
    print(f'포트폴리오의 표준편차는: {round(STD(pfo_ret),4)}')
    
    r_f = 0.02
    print(f'포트폴리오의 Sharpe Ratio는: {round(SHARPE(pfo_ret, r_f),2)}')

    return cum_ret

def cum_ret_gpt(dic, start_date, end_date):  

    data = download(dic, start_date, end_date)
    ret = data.pct_change().dropna()
    pfo_ret = ret.dot(pd.Series(dic))
    cum_ret = (1 + pfo_ret).cumprod()

    print(f'포트폴리오의 MDD는: {round(MDD(cum_ret),2)*100}%')
    print(f'포트폴리오의 표준편차는: {round(STD(pfo_ret),4)}')
    
    r_f = 0.02
    print(f'포트폴리오의 Sharpe Ratio는: {round(SHARPE(pfo_ret, r_f),2)}')
        
    return cum_ret

In [39]:
# GMV, GPT OUT OF SAMPLE

gmv_15_out = cum_ret_gmv(stocks_15, gmv_15, '2023-05-01', '2023-10-10')
gmv_30_out = cum_ret_gmv(stocks_30, gmv_30, '2023-05-01', '2023-10-10')
gmv_45_out = cum_ret_gmv(stocks_45, gmv_45, '2023-05-01', '2023-10-10')

gpt_15_out = cum_ret_gpt(stocks_15, '2023-05-01', '2023-10-10')
gpt_30_out = cum_ret_gpt(stocks_30, '2023-05-01', '2023-10-10')
gpt_45_out = cum_ret_gpt(stocks_45, '2023-05-01', '2023-10-10')

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 616.17it/s]


포트폴리오의 MDD는: 11.0%
포트폴리오의 표준편차는: 0.0062
포트폴리오의 Sharpe Ratio는: -3.27


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 582.98it/s]


포트폴리오의 MDD는: 9.0%
포트폴리오의 표준편차는: 0.0058
포트폴리오의 Sharpe Ratio는: -3.5


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 641.95it/s]


...

포트폴리오의 MDD는: 8.0%
포트폴리오의 표준편차는: 0.008
포트폴리오의 Sharpe Ratio는: -2.42


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 663.95it/s]


포트폴리오의 MDD는: 8.0%
포트폴리오의 표준편차는: 0.0079
포트폴리오의 Sharpe Ratio는: -2.44


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 707.72it/s]

포트폴리오의 MDD는: 7.000000000000001%
포트폴리오의 표준편차는: 0.0069
포트폴리오의 Sharpe Ratio는: -2.85


In [40]:
# SAME WEIGHTS IN, OUT OF SAMPLE

gpt_15_samein = cum_ret_gpt(gpt15_same, '2018-05-01', '2023-10-31')
gpt_30_samein = cum_ret_gpt(gpt30_same, '2018-05-01', '2023-10-31')
gpt_45_samein = cum_ret_gpt(gpt45_same, '2018-05-01', '2023-10-31')

gpt_15_sameout = cum_ret_gpt(gpt15_same, '2023-05-01', '2023-10-31')
gpt_30_sameout = cum_ret_gpt(gpt30_same, '2023-05-01', '2023-10-31')
gpt_45_sameout = cum_ret_gpt(gpt45_same, '2023-05-01', '2023-10-31')

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 289.83it/s]


포트폴리오의 MDD는: 34.0%
포트폴리오의 표준편차는: 0.0139
포트폴리오의 Sharpe Ratio는: -1.38


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 309.02it/s]


포트폴리오의 MDD는: 33.0%
포트폴리오의 표준편차는: 0.0134
포트폴리오의 Sharpe Ratio는: -1.44


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 200.38it/s]


...

포트폴리오의 MDD는: 9.0%
포트폴리오의 표준편차는: 0.0077
포트폴리오의 Sharpe Ratio는: -2.54


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 686.45it/s]


포트폴리오의 MDD는: 8.0%
포트폴리오의 표준편차는: 0.007
포트폴리오의 Sharpe Ratio는: -2.81


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 597.44it/s]

포트폴리오의 MDD는: 7.000000000000001%
포트폴리오의 표준편차는: 0.0067
포트폴리오의 Sharpe Ratio는: -2.92


In [41]:
# 동일가중

def make_dict(dict):
    dic = {}
    
    for column in dict.keys():
        dic[column] = 1/len(dict.keys())

    return dic
    
gpt15_same = make_dict(stocks_15)
gpt30_same = make_dict(stocks_30)
gpt45_same = make_dict(stocks_45)

In [42]:
# S&P500 Insample 기간

snpin = yf.download('^GSPC', '2018-05-01', '2023-05-01')['Adj Close']
snp_ret_in = snpin.pct_change().dropna()
snpin = (1 + snp_ret_in).cumprod()

# S&P500 Out of Sample 기간
snpout = yf.download('^GSPC', '2023-05-01', '2023-10-31')['Adj Close']
snp_ret_out = snpout.pct_change().dropna()
snpout = (1 + snp_ret_out).cumprod()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [43]:
# GMV, GPT IN SAMPLE

gmv_15_in = cum_ret_gmv(stocks_15, gmv_15, '2018-05-01', '2023-10-10')
gmv_30_in = cum_ret_gmv(stocks_30, gmv_30, '2018-05-01', '2023-10-10')
gmv_45_in = cum_ret_gmv(stocks_45, gmv_45, '2018-05-01', '2023-10-10')

gpt_15_in = cum_ret_gpt(stocks_15, '2018-05-01', '2023-10-10')
gpt_30_in = cum_ret_gpt(stocks_30, '2018-05-01', '2023-10-10')
gpt_45_in = cum_ret_gpt(stocks_45, '2018-05-01', '2023-10-10')

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 261.54it/s]


포트폴리오의 MDD는: 31.0%
포트폴리오의 표준편차는: 0.0117
포트폴리오의 Sharpe Ratio는: -1.66


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 334.82it/s]


포트폴리오의 MDD는: 28.999999999999996%
포트폴리오의 표준편차는: 0.0112
포트폴리오의 Sharpe Ratio는: -1.73


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 321.79it/s]


...

포트폴리오의 MDD는: 33.0%
포트폴리오의 표준편차는: 0.0142
포트폴리오의 Sharpe Ratio는: -1.34


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 322.09it/s]


포트폴리오의 MDD는: 31.0%
포트폴리오의 표준편차는: 0.0139
포트폴리오의 Sharpe Ratio는: -1.38


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 319.54it/s]

포트폴리오의 MDD는: 33.0%
포트폴리오의 표준편차는: 0.0132
포트폴리오의 Sharpe Ratio는: -1.46


## **Max Sharpe Ratio**

In [44]:
def max_sharpe(dict):
    df = download(dict, "2018-05-01", "2023-04-30")

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    ef = EfficientFrontier(mu, S)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets

    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)
    
    ef.add_constraint(lambda w: w.sum() == 1)

    w_opt = ef.max_sharpe() # Max Sharpe ratio가 되는 weights 찾기

    w_clean = ef.clean_weights()
    print()
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

    print()
    print("가장 큰 가중치를 가진 자산:", max_weight_asset)
    print("가장 큰 가중치:", max_weight_value)

    return w_clean

In [45]:
msp_15 = max_sharpe(stocks_15)
msp_30 = max_sharpe(stocks_30)
msp_45 = max_sharpe(stocks_45)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 301.61it/s]



Expected annual return: 24.8%
Annual volatility: 24.9%
Sharpe Ratio: 0.92

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.13333


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 351.56it/s]



Expected annual return: 19.8%
Annual volatility: 22.0%
Sharpe Ratio: 0.81

가장 큰 가중치를 가진 자산: MSFT
가장 큰 가중치: 0.06667


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 347.71it/s]



Expected annual return: 19.4%
Annual volatility: 21.2%
Sharpe Ratio: 0.82

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.04444


In [46]:
def max_sharpe_cum_ret(dic, result, start_date, end_date):
    data = download(dic, start_date, end_date)
    ret = data.pct_change().dropna()
    pfo_ret = ret.dot(pd.Series(result))
    cum_ret = (1 + pfo_ret).cumprod()
    
    print(f'포트폴리오의 MDD는: {round(MDD(cum_ret),2)*100}%')
    print(f'포트폴리오의 표준편차는: {round(STD(pfo_ret),4)}')
    
    r_f = 0.02
    print(f'포트폴리오의 Sharpe Ratio는: {round(SHARPE(pfo_ret, r_f),2)}')

    return cum_ret

msf_15_out = max_sharpe_cum_ret(stocks_15, msp_15, '2023-05-01', '2023-10-10')
msf_30_out = max_sharpe_cum_ret(stocks_30, msp_30, '2023-05-01', '2023-10-10')
msf_45_out = max_sharpe_cum_ret(stocks_45, msp_45, '2023-05-01', '2023-10-10')

msf_15_in = max_sharpe_cum_ret(stocks_15, msp_15, '2018-05-01', '2023-10-10')
msf_30_in = max_sharpe_cum_ret(stocks_30, msp_30, '2018-05-01', '2023-10-10')
msf_45_in = max_sharpe_cum_ret(stocks_45, msp_45, '2018-05-01', '2023-10-10')

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 625.81it/s]


포트폴리오의 MDD는: 10.0%
포트폴리오의 표준편차는: 0.0091
포트폴리오의 Sharpe Ratio는: -2.12


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 696.62it/s]


포트폴리오의 MDD는: 8.0%
포트폴리오의 표준편차는: 0.0072
포트폴리오의 Sharpe Ratio는: -2.7


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 733.16it/s]


...

포트폴리오의 MDD는: 33.0%
포트폴리오의 표준편차는: 0.0152
포트폴리오의 Sharpe Ratio는: -1.24


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 183.70it/s]


포트폴리오의 MDD는: 31.0%
포트폴리오의 표준편차는: 0.0134
포트폴리오의 Sharpe Ratio는: -1.42


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 313.42it/s]


포트폴리오의 MDD는: 30.0%
포트폴리오의 표준편차는: 0.0129
포트폴리오의 Sharpe Ratio는: -1.48


## **Max Expected Returns**

In [47]:
def max_return(dict):
    df = download(dict, "2018-05-01", "2023-04-30")

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    ef = EfficientFrontier(mu, S, solver='ECOS')
    ef.add_constraint(lambda x: x.sum() == 1,)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets
    
    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)

    w_opt = ef._max_return()

    w_clean = ef.clean_weights()
    print()
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

    print()
    print("가장 큰 가중치를 가진 자산:", max_weight_asset)
    print("가장 큰 가중치:", max_weight_value)

    return w_clean

In [48]:
max_ret_15 = max_return(stocks_15)
max_ret_30 = max_return(stocks_30)
max_ret_45 = max_return(stocks_45)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 325.15it/s]



Expected annual return: 26.3%
Annual volatility: 26.7%
Sharpe Ratio: 0.91

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.13333


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 343.09it/s]



Expected annual return: 20.3%
Annual volatility: 23.4%
Sharpe Ratio: 0.78

가장 큰 가중치를 가진 자산: MSFT
가장 큰 가중치: 0.06667


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 342.81it/s]



Expected annual return: 19.6%
Annual volatility: 22.1%
Sharpe Ratio: 0.80

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.04444


In [49]:
def max_sharpe_cum_ret(dic, result, start_date, end_date):
    data = download(dic, start_date, end_date)
    ret = data.pct_change().dropna()
    pfo_ret = ret.dot(pd.Series(result))
    cum_ret = (1 + pfo_ret).cumprod()

    print(f'포트폴리오의 MDD는: {round(MDD(cum_ret),2)*100}%')
    print(f'포트폴리오의 표준편차는: {round(STD(pfo_ret),4)}')
    
    r_f = 0.02
    print(f'포트폴리오의 Sharpe Ratio는: {round(SHARPE(pfo_ret, r_f),2)}')

    return cum_ret

max_15_out = max_sharpe_cum_ret(stocks_15, max_ret_15, '2023-05-01', '2023-10-10')
max_30_out = max_sharpe_cum_ret(stocks_30, max_ret_30, '2023-05-01', '2023-10-10')
max_45_out = max_sharpe_cum_ret(stocks_45, max_ret_45, '2023-05-01', '2023-10-10')

max_15_in = max_sharpe_cum_ret(stocks_15, max_ret_15, '2018-05-01', '2023-10-10')
max_30_in = max_sharpe_cum_ret(stocks_30, max_ret_30, '2018-05-01', '2023-10-10')
max_45_in = max_sharpe_cum_ret(stocks_45, max_ret_45, '2018-05-01', '2023-10-10')

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 669.80it/s]


포트폴리오의 MDD는: 9.0%
포트폴리오의 표준편차는: 0.0105
포트폴리오의 Sharpe Ratio는: -1.79


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 694.91it/s]


포트폴리오의 MDD는: 8.0%
포트폴리오의 표준편차는: 0.0079
포트폴리오의 Sharpe Ratio는: -2.44


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 716.50it/s]


...

포트폴리오의 MDD는: 33.0%
포트폴리오의 표준편차는: 0.0164
포트폴리오의 Sharpe Ratio는: -1.14


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 338.97it/s]


포트폴리오의 MDD는: 32.0%
포트폴리오의 표준편차는: 0.0143
포트폴리오의 Sharpe Ratio는: -1.33


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 327.63it/s]

포트폴리오의 MDD는: 31.0%
포트폴리오의 표준편차는: 0.0135
포트폴리오의 Sharpe Ratio는: -1.42


## **Rebalancing**

In [50]:
nasdaq_out = yf.download('^IXIC', '2023-05-01', '2023-11-01')['Adj Close']
nasdaq_ret_out = nasdaq_out.pct_change().dropna()
nasdaq_rb = (1 + nasdaq_ret_out).cumprod()

[*********************100%%**********************]  1 of 1 completed


In [51]:
# 초기 누적수익률 확인한 날짜까지 최적화에 포함해서 리밸런싱 진행

def gmv_rb(dict):

    data = download(dict, "2018-05-01", "2023-08-01")
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    cov_mat = np.cov(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Covariance Matrix를 만듬

    print(cov_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, cov_mat)) # 목적식 설정
    
    # weights >= 0 
    const = [cp.sum(weights) == 1, weights >= 0] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt

def max_sharpe_rb(dict):
    df = download(dict, "2018-05-01", "2023-08-01")

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    ef = EfficientFrontier(mu, S)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets

    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)
    
    ef.add_constraint(lambda w: w.sum() == 1)

    w_opt = ef.max_sharpe() # Max Sharpe ratio가 되는 weights 찾기

    w_clean = ef.clean_weights()
    print()
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

    print()
    print("가장 큰 가중치를 가진 자산:", max_weight_asset)
    print("가장 큰 가중치:", max_weight_value)

    return w_clean

def max_return_rb(dict):
    df = download(dict, "2018-05-01", "2023-08-01")

    mu = expected_returns.mean_historical_return(df) # pfo mean 
    S = risk_models.sample_cov(df) # cov
    ef = EfficientFrontier(mu, S, solver='ECOS')
    ef.add_constraint(lambda x: x.sum() == 1,)

    n_assets = len(mu)  # 자산 수
    min_weights = 0.5 * 1 / n_assets  # 각 자산별 최소 weight
    max_weights = 2 * 1 / n_assets
    
    for asset in mu.index:
        num = mu.index.get_loc(asset)
        ef.add_constraint(lambda w: w[num] <= max_weights)
        ef.add_constraint(lambda w: w[num] >= min_weights)

    w_opt = ef._max_return()

    w_clean = ef.clean_weights()
    print()
    ef.portfolio_performance(verbose=True) 
    # print(w_clean) # 가중치 확인
    max_weight_asset = max(w_clean, key=w_clean.get)
    max_weight_value = w_clean[max_weight_asset]

    print()
    print("가장 큰 가중치를 가진 자산:", max_weight_asset)
    print("가장 큰 가중치:", max_weight_value)

    return w_clean

gpt_15_rbw = gmv_rb(stocks_15)
gpt_30_rbw = gmv_rb(stocks_30)
gpt_45_rbw = gmv_rb(stocks_45)

msp_15_rbw = max_sharpe_rb(stocks_15)
msp_30_rbw = max_sharpe_rb(stocks_30)
msp_45_rbw = max_sharpe_rb(stocks_45)

maxret_15_rbw = max_return_rb(stocks_15)
maxret_30_rbw = max_return_rb(stocks_30)
maxret_45_rbw = max_return_rb(stocks_45)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 272.74it/s]

[[4.24626687e-04 3.00825286e-04 4.42299250e-04 1.89242325e-04
  1.17388910e-04 1.12653376e-04 1.05517386e-04 1.13673963e-04
  1.45763410e-04 1.48283204e-04 2.81052404e-04 3.02601760e-04
  4.00948592e-04 2.28160173e-04 2.09273414e-04]
 [3.00825286e-04 5.08604076e-04 4.59377325e-04 1.36816748e-04
  8.01567298e-05 8.36481929e-05 7.14538835e-05 7.12906145e-05
  9.63642002e-05 1.15073142e-04 3.05362353e-04 3.09056271e-04
  3.97314029e-04 2.15170696e-04 1.62762210e-04]
 [4.42299250e-04 4.59377325e-04 1.10294448e-03 2.50058398e-04
  1.23378367e-04 1.20703410e-04 9.97214232e-05 1.13370416e-04
  1.82375042e-04 1.82607290e-04 4.17615317e-04 4.48714327e-04
  6.51868409e-04 3.38885975e-04 2.82787444e-04]
 [1.89242325e-04 1.36816748e-04 2.50058398e-04 3.99380544e-04
  1.03184451e-04 1.20653812e-04 1.03945224e-04 1.40652563e-04
  2.51952052e-04 1.25433681e-04 1.79017690e-04 1.77297609e-04
  2.24533552e-04 2.07706367e-04 4.05519185e-04]
 [1.17388910e-04 8.01567298e-05 1.23378367e-04 1.03184451e-04
  

Optimal


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 328.07it/s]

[[3.75330184e-04 3.09056271e-04 4.48714327e-04 3.02601760e-04
  3.00861042e-04 3.59864544e-04 1.04989090e-04 1.14910082e-04
  1.06201579e-04 1.92031589e-04 1.21260237e-04 1.12644457e-04
  1.77297609e-04 2.03245844e-04 1.61602062e-04 1.61437095e-04
  1.25973279e-04 2.45516519e-04 3.59626963e-04 1.51621946e-04
  2.18844930e-04 7.30716409e-05 3.27915978e-04 1.81288433e-04
  1.10646804e-04 3.00015624e-04 1.98378680e-04 2.33079676e-04
  1.38522975e-04 2.79703227e-04]
 [3.09056271e-04 5.08604076e-04 4.59377325e-04 3.00825286e-04
  3.05362353e-04 3.52676663e-04 7.14538835e-05 8.36481929e-05
  6.40615372e-05 1.59898747e-04 8.01567298e-05 7.12906145e-05
  1.36816748e-04 1.80123516e-04 1.39687594e-04 1.06404723e-04
  9.63642002e-05 2.15209835e-04 3.97314029e-04 1.15073142e-04
  2.15170696e-04 5.36719067e-05 3.41765370e-04 1.28617158e-04
  9.95247479e-05 2.92041230e-04 1.62762210e-04 1.97596334e-04
  9.33122092e-05 2.40815966e-04]
 [4.48714327e-04 4.59377325e-04 1.10294448e-03 4.42299250e-04
  4.

...


Expected annual return: 29.6%
Annual volatility: 27.8%
Sharpe Ratio: 0.99

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.13333


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 338.85it/s]



Expected annual return: 22.4%
Annual volatility: 24.1%
Sharpe Ratio: 0.85

가장 큰 가중치를 가진 자산: MSFT
가장 큰 가중치: 0.06667


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 309.89it/s]



Expected annual return: 21.2%
Annual volatility: 21.8%
Sharpe Ratio: 0.88

가장 큰 가중치를 가진 자산: AAPL
가장 큰 가중치: 0.04444


In [52]:
gpt_15_rbrt = cum_gmv_rb(stocks_15, gpt_15_rbw, '2023-08-01', '2023-11-01')
gpt_30_rbrt = cum_gmv_rb(stocks_30, gpt_30_rbw, '2023-08-01', '2023-11-01')
gpt_45_rbrt = cum_gmv_rb(stocks_45, gpt_45_rbw, '2023-08-01', '2023-11-01')

msp_15_rbrt = cum_rb(stocks_15, msp_15_rbw, '2023-08-01', '2023-11-01')
msp_30_rbrt = cum_rb(stocks_30, msp_30_rbw, '2023-08-01', '2023-11-01')
msp_45_rbrt = cum_rb(stocks_45, msp_45_rbw, '2023-08-01', '2023-11-01')

maxret_15_rbrt = cum_rb(stocks_15, maxret_15_rbw, '2023-08-01', '2023-11-01')
maxret_30_rbrt = cum_rb(stocks_30, maxret_30_rbw, '2023-08-01', '2023-11-01')
maxret_45_rbrt = cum_rb(stocks_45, maxret_45_rbw, '2023-08-01', '2023-11-01')

NameError: name 'cum_gmv_rb' is not defined

In [ ]:
gpt_15_rb = pd.concat([gpt_15_rb, gpt_15_rbrt])
gpt_30_rb = pd.concat([gpt_30_rb, gpt_30_rbrt])
gpt_45_rb = pd.concat([gpt_45_rb, gpt_45_rbrt])

msp_15_rb = pd.concat([msp_15_rb, msp_15_rbrt])
msp_30_rb = pd.concat([msp_30_rb, msp_30_rbrt])
msp_45_rb = pd.concat([msp_45_rb, msp_45_rbrt])

maxret_15_rb = pd.concat([maxret_15_rb, maxret_15_rbrt])
maxret_30_rb = pd.concat([maxret_30_rb, maxret_30_rbrt])
maxret_45_rb = pd.concat([maxret_45_rb, maxret_45_rbrt])

In [ ]:
# SHARPE와 GPT종목 same weights랑, Market Index랑 비교 Out-sample

plt.figure(figsize=(10,6))
plt.title("GMV REBALANCING")

gpt_15_rb.plot(label='GPT15', color='red')
gpt_30_rb.plot(label='GPT30', color='green')
gpt_45_rb.plot(label='GPT45', color='blue')

nasdaq_rb.plot(label='NASDAQ', color='orange')

plt.legend()
plt.show()

In [ ]:
# SHARPE와 GPT종목 same weights랑, Market Index랑 비교 Out-sample

plt.figure(figsize=(10,6))
plt.title("MAX SHARPE REBALANCING")

msp_15_rb.plot(label='MAX SHARPE 15', color='red')
msp_30_rb.plot(label='MAX SHARPE 30', color='green')
msp_45_rb.plot(label='MAX SHARPE 45', color='blue')

nasdaq_rb.plot(label='NASDAQ', color='orange')

plt.legend()
plt.show()

In [ ]:
# SHARPE와 GPT종목 same weights랑, Market Index랑 비교 Out-sample

plt.figure(figsize=(10,6))
plt.title("MAX RETURN REBALANCING")

maxret_15_rb.plot(label='MAX RETURN 15', color='red')
maxret_30_rb.plot(label='MAX RETURN 30', color='green')
maxret_45_rb.plot(label='MAX RETURN 45', color='blue')

nasdaq_rb.plot(label='NASDAQ', color='orange')

plt.legend()
plt.show()

## LOOK BACK

In [ ]:
# GMV 함수

def gmv_lookback(dict, start_date, end_date):

    data = download(dict, start_date, end_date)
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    cov_mat = np.cov(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Covariance Matrix를 만듬

    print(cov_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, cov_mat)) # 목적식 설정
    
    # weights >= 0 
    const = [cp.sum(weights) == 1, weights >= 0] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt

In [ ]:
gmv_15 = gmv_lookback(stocks_15, '2018-05-01', '2023-06-01')
gmv_30 = gmv_lookback(stocks_30, '2018-05-01', '2023-06-01')
gmv_45 = gmv_lookback(stocks_45, '2018-05-01', '2023-06-01')

gmv_1year_15 = gmv_lookback(stocks_15, '2022-05-01', '2023-06-01')
gmv_1year_30 = gmv_lookback(stocks_30, '2022-05-01', '2023-06-01')
gmv_1year_45 = gmv_lookback(stocks_45, '2022-05-01', '2023-06-01')

gmv_2year_15 = gmv_lookback(stocks_15, '2021-05-01', '2023-06-01')
gmv_2year_30 = gmv_lookback(stocks_30, '2021-05-01', '2023-06-01')
gmv_2year_45 = gmv_lookback(stocks_45, '2021-05-01', '2023-06-01')

In [ ]:
gmv_15_ret = cum_gmv_rb(stocks_15, gmv_15, '2023-06-01', '2023-11-01')
gmv_30_ret = cum_gmv_rb(stocks_30, gmv_30, '2023-06-01', '2023-11-01')
gmv_45_ret = cum_gmv_rb(stocks_45, gmv_45, '2023-06-01', '2023-11-01')

gmv_1year_ret15 = cum_gmv_rb(stocks_15, gmv_1year_15, '2023-06-01', '2023-11-01')
gmv_1year_ret30 = cum_gmv_rb(stocks_30, gmv_1year_30, '2023-06-01', '2023-11-01')
gmv_1year_ret45 = cum_gmv_rb(stocks_45, gmv_1year_45, '2023-06-01', '2023-11-01')

gmv_2year_ret15 = cum_gmv_rb(stocks_15, gmv_2year_15, '2023-06-01', '2023-11-01')
gmv_2year_ret30 = cum_gmv_rb(stocks_30, gmv_2year_30, '2023-06-01', '2023-11-01')
gmv_2year_ret45 = cum_gmv_rb(stocks_45, gmv_2year_45, '2023-06-01', '2023-11-01')

In [ ]:
# SHARPE와 GPT종목 same weights랑, Market Index랑 비교 Out-sample

plt.figure(figsize=(10,6))
plt.title("MAX RETURN REBALANCING")

gmv_15_ret.plot(label='GMV FULL REBLANCE 15', color='red')
gmv_30_ret.plot(label='GMV FULL REBLANCE 30', color='green')
gmv_45_ret.plot(label='GMV FULL REBLANCE 45', color='blue')

gmv_1year_ret15.plot(label='1YEAR LOOKBACK 15', color='black')
gmv_1year_ret30.plot(label='1YEAR LOOKBACK 30', color='yellow')
gmv_1year_ret45.plot(label='1YEAR LOOKBACK 45', color='pink')

gmv_2year_ret15.plot(label='2YEAR LOOKBACK 15', color='orange')
gmv_2year_ret30.plot(label='2YEAR LOOKBACK 30', color='purple')
gmv_2year_ret45.plot(label='2YEAR LOOKBACK 45', color='gray')

plt.legend()
plt.show()

## Other Comparison

In [ ]:
import statsmodels.api as sm
import numpy as np

In [ ]:
# 전체 기간 기본 모멘텀
# 티커별 수익률로 환산

ret = market_data.iloc[-1]/market_data.iloc[0]-1
ret_df = pd.DataFrame({'ticker': ret.index, 'return': ret})
ret_df = ret_df.reset_index(drop=True)

In [ ]:
# 수익률 상위 20개 종목 추출

ret_rank = ret_df['return'].rank(axis=0, ascending=False)
rank_df = ret_df[ret_rank <= 15]
rank_tickers = rank_df['ticker']

In [ ]:
# 전체 기간 K-ratio

data_ret = market_data.pct_change().iloc[1:]
x = np.array(range(len(data_ret)))
k_ratio = {}

for i, column in enumerate(data_ret.columns):
    y = data_cum.iloc[:, i].values

    reg = sm.OLS(y, x).fit()
    result = float((reg.params/reg.bse).item())

    k_ratio[column] = result

k_ratio = pd.DataFrame.from_dict(k_ratio, orient='index').reset_index()
k_ratio.columns = ['ticker', 'K_ratio']

krat_df = ret_df.merge(k_ratio, how='left', on='ticker')
krat_df_rank = krat_df['K_ratio'].rank(axis=0, ascending=False)
krat_df_15 = krat_df[krat_df_rank <= 15]
krat_15_tickers = krat_df_15['ticker']
krat_df_30 = krat_df[krat_df_rank <= 30]
krat_30_tickers = krat_df_30['ticker']
krat_df_45 = krat_df[krat_df_rank <= 45]
krat_45_tickers = krat_df_45['ticker']

In [ ]:
# 전체 기간 상위 20개 기본 모멘텀
mmt_data = market_data[rank_tickers]

num_plots = len(mmt_data.columns)
num_cols = 3
num_rows = (num_plots // num_cols)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))
fig.subplots_adjust(hspace=0.5)

for i, (col, ax) in enumerate(zip(mmt_data.columns, axes.flatten())):
    mmt_data[col].plot(ax=ax)
    ax.set_title(col)
    ax.set_xlabel('Date')
    ax.set_ylabel('Adj Close')

for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 전체 기간 상위 20개 K-ratio

krat_data = market_data[krat_20_tickers]

num_plots = len(krat_data.columns)
num_cols = 3
num_rows = (num_plots // num_cols)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))
fig.subplots_adjust(hspace=0.5)

for i, (col, ax) in enumerate(zip(krat_data.columns, axes.flatten())):
    krat_data[col].plot(ax=ax)
    ax.set_title(col)
    ax.set_xlabel('Date')
    ax.set_ylabel('Adj Close')

for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

plt.tight_layout()
plt.show()